In [37]:
import mysql.connector
import pandas as pd
from joblib import load

In [38]:
# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='127.0.0.1',
    database='djezzy',
    user='root',
    password=''
)

# Check if the connection is successfully established
if connection.is_connected():
    print("Connected to MySQL database")
else:
    print(f"Error connecting to MySQL database")

Connected to MySQL database


In [39]:
# Read data from MySQL database table 'c_d_r_s' into a pandas DataFrame
df = pd.read_sql("SELECT * FROM c_d_r_s", connection)

# Display the first few rows of the DataFrame
df.head()

C:\Users\tr3h2\AppData\Local\Temp\ipykernel_20836\570282810.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM c_d_r_s", connection)


,id,A_Num,Call_Type,Charging_Tm,Call_Duration,Telesrvc,LOCATION,B_Num,cell_id,IMEI,TAC,DESTINATION_CAT,MU_HANDSET_DUAL_SIM,MU_Device_type_Segment,MU_HANDSET_MOBILE_TECH,created_at,updated_at
0,1,12751,6,2024-04-14 12:31:13,2.000000,22,4667,97127,7504492,398,297,ON-NET,Y,Smartphone,4G,2024-05-24 10:23:09,2024-05-24 10:23:09
1,2,11295,6,2024-04-18 14:33:27,2.000000,22,18422,97127,7587539,6217,688,ON-NET,Y,Smartphone,4G,2024-05-24 10:23:09,2024-05-24 10:23:09
2,3,14819,6,2024-04-24 15:57:56,2.000000,22,34742,97127,7449361,6080,583,ON-NET,Y,Smartphone,4G,2024-05-24 10:23:09,2024-05-24 10:23:09
3,4,4717,1,2024-04-15 17:43:00,25.535412,11,36304,208604,7440187,14089,4545,ON-NET,Y,Smartphone,4G,2024-05-24 10:23:09,2024-05-24 10:23:09
4,5,15965,0,2024-04-18 22:17:45,17.023608,11,44589,32614,7453014,16523,5130,OFF-NET,Y,Smartphone,4G,2024-05-24 10:23:09,2024-05-24 10:23:09


In [40]:
# Path to the machine learning model file
model = "models/XGBClassifier.joblib"

# Load the machine learning model
model = load(model)

In [41]:
features_df = pd.DataFrame()
# Calculating the mean and standard deviation of the time differences between consecutive "Charging_Tm" values for each "A_Num" group.
features_df[['AVG_CALL_FREQ', 'STD_CALL_FREQ']] = df.sort_values(['A_Num', 'Charging_Tm']).groupby('A_Num')['Charging_Tm'].diff().dt.total_seconds().groupby(df.sort_values(['A_Num', 'Charging_Tm'])['A_Num']).agg(['mean', 'std'])


# Creating features related to destination categories ("ON-NET", "OFF-NET", "INTERNATIONAL"):
for des in ["ON-NET", "OFF-NET", "INTERNATIONAL"]:
   features_df[des.replace('-', '_')+'_RATIO'] = df.groupby('A_Num').agg(ON_NET_RATIO=('DESTINATION_CAT', lambda x: (x==des).sum() / len(x)))
# Creating features related to call types (0, 1, 6, 7):
for call_type in [0, 1, 6, 7]:
    features_df['CALLTYPE_'+str(call_type)+'_RATIO'] = df.groupby('A_Num').agg(TELESRVC11_RATIO=('Call_Type', lambda x: (x==call_type).sum() / len(x)))

# Creating features related to call frequency:
features_df['MIN_CALL_FREQ'] = df.groupby('A_Num').apply(lambda x: x.sort_values('Charging_Tm')).reset_index(drop=True).assign(INTERVAL_ACTIVITIES_SECONDS=lambda x: x.groupby('A_Num')['Charging_Tm'].diff().dt.total_seconds()).groupby('A_Num')['INTERVAL_ACTIVITIES_SECONDS'].min()
features_df['AVG_CALL_FREQ'] = df.groupby('A_Num').apply(lambda x: x.sort_values('Charging_Tm')).reset_index(drop=True).assign(INTERVAL_ACTIVITIES_SECONDS=lambda x: x.groupby('A_Num')['Charging_Tm'].diff().dt.total_seconds()).groupby('A_Num')['INTERVAL_ACTIVITIES_SECONDS'].mean()
features_df['MAX_CALL_FREQ'] = df.groupby('A_Num').apply(lambda x: x.sort_values('Charging_Tm')).reset_index(drop=True).assign(INTERVAL_ACTIVITIES_SECONDS=lambda x: x.groupby('A_Num')['Charging_Tm'].diff().dt.total_seconds()).groupby('A_Num')['INTERVAL_ACTIVITIES_SECONDS'].max()

# Creating features related to daily call counts and durations:
features_df['MIN_DAILY_CALLS'] = df.groupby(['A_Num', df['Charging_Tm'].dt.date])['A_Num'].count().groupby('A_Num').min()
features_df['AVG_DAILY_CALLS'] = df.groupby(['A_Num', df['Charging_Tm'].dt.date])['A_Num'].count().groupby('A_Num').mean()
features_df['MAX_DAILY_CALLS'] = df.groupby(['A_Num', df['Charging_Tm'].dt.date])['A_Num'].count().groupby('A_Num').max()
features_df['MIN_DAILY_CALLS_DUR'] = df.groupby(['A_Num', df['Charging_Tm'].dt.date])['Call_Duration'].sum().groupby('A_Num').min()
features_df['AVG_DAILY_CALLS_DUR'] = df.groupby(['A_Num', df['Charging_Tm'].dt.date])['Call_Duration'].sum().groupby('A_Num').mean()
features_df['MAX_DAILY_CALLS_DUR'] = df.groupby(['A_Num', df['Charging_Tm'].dt.date])['Call_Duration'].sum().groupby('A_Num').max()

features_df['A_Num'] = features_df.index
# Displaying the first few rows of the resulting features DataFrame features_df.
features_df.head()

C:\Users\tr3h2\AppData\Local\Temp\ipykernel_20836\2650195964.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features_df['MIN_CALL_FREQ'] = df.groupby('A_Num').apply(lambda x: x.sort_values('Charging_Tm')).reset_index(drop=True).assign(INTERVAL_ACTIVITIES_SECONDS=lambda x: x.groupby('A_Num')['Charging_Tm'].diff().dt.total_seconds()).groupby('A_Num')['INTERVAL_ACTIVITIES_SECONDS'].min()
C:\Users\tr3h2\AppData\Local\Temp\ipykernel_20836\2650195964.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude

,AVG_CALL_FREQ,STD_CALL_FREQ,ON_NET_RATIO,OFF_NET_RATIO,INTERNATIONAL_RATIO,CALLTYPE_0_RATIO,CALLTYPE_1_RATIO,CALLTYPE_6_RATIO,CALLTYPE_7_RATIO,MIN_CALL_FREQ,MAX_CALL_FREQ,MIN_DAILY_CALLS,AVG_DAILY_CALLS,MAX_DAILY_CALLS,MIN_DAILY_CALLS_DUR,AVG_DAILY_CALLS_DUR,MAX_DAILY_CALLS_DUR,A_Num
A_Num,,,,,,,,,,,,,,,,,,
455,116857.0,136700.737392,0.666667,0.333333,0.0,0.666667,0.333333,0.000000,0.000000,4733.0,354485.0,1,1.2,2,39.721752,70.553398,158.887008,455
1355,NaN,NaN,0.000000,1.000000,0.0,0.000000,1.000000,0.000000,0.000000,NaN,NaN,1,1.0,1,43.504776,43.504776,43.504776,1355
1384,64659.8,59350.342237,0.666667,0.000000,0.0,0.000000,0.000000,0.666667,0.333333,4360.0,151301.0,1,1.5,3,2.000000,3.000000,6.000000,1384
1404,240540.0,54131.458100,0.250000,0.750000,0.0,0.500000,0.500000,0.000000,0.000000,178150.0,275026.0,1,1.0,1,17.969364,40.194630,65.257164,1404
1487,NaN,NaN,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,NaN,NaN,1,1.0,1,74.714724,74.714724,74.714724,1487


In [42]:
# Filling missing values in the features DataFrame features_df with zeros
features_df = features_df.fillna(0)

In [43]:
# Getting the names of all columns in the DataFrame features_df except for 'A_Num' and 'is_fraud'
features = features_df.columns.difference(['A_Num', 'is_fraud'])

features

Index(['AVG_CALL_FREQ', 'AVG_DAILY_CALLS', 'AVG_DAILY_CALLS_DUR',
       'CALLTYPE_0_RATIO', 'CALLTYPE_1_RATIO', 'CALLTYPE_6_RATIO',
       'CALLTYPE_7_RATIO', 'INTERNATIONAL_RATIO', 'MAX_CALL_FREQ',
       'MAX_DAILY_CALLS', 'MAX_DAILY_CALLS_DUR', 'MIN_CALL_FREQ',
       'MIN_DAILY_CALLS', 'MIN_DAILY_CALLS_DUR', 'OFF_NET_RATIO',
       'ON_NET_RATIO', 'STD_CALL_FREQ'],
      dtype='object')

In [44]:
from sklearn.preprocessing import PowerTransformer

# Initializing PowerTransformer with 'yeo-johnson' method for handling both positive and negative values
# Setting standardize=True to center and scale the data
pt = PowerTransformer(method='yeo-johnson', standardize=True)

# Drop 'A_Num' column temporarily for transformation
features_for_transformation = features_df.drop(columns=["A_Num"])

# Fit and transform the data
transformed_features = pt.fit_transform(features_for_transformation)

# Creating DataFrame from the transformed features with column names
transformed_df = pd.DataFrame(transformed_features, columns=features_for_transformation.columns)
# Add the 'A_Num' column back to the DataFrame
transformed_df["A_Num"] = features_df["A_Num"].values.reshape(-1, 1)

transformed_df

,AVG_CALL_FREQ,STD_CALL_FREQ,ON_NET_RATIO,OFF_NET_RATIO,INTERNATIONAL_RATIO,CALLTYPE_0_RATIO,CALLTYPE_1_RATIO,CALLTYPE_6_RATIO,CALLTYPE_7_RATIO,MIN_CALL_FREQ,MAX_CALL_FREQ,MIN_DAILY_CALLS,AVG_DAILY_CALLS,MAX_DAILY_CALLS,MIN_DAILY_CALLS_DUR,AVG_DAILY_CALLS_DUR,MAX_DAILY_CALLS_DUR,A_Num
0,1.282615,1.506852,-0.215932,1.405131,-0.262983,0.850882,1.334296,-0.604219,-0.554667,1.225962,1.286112,-0.587926,-0.306012,0.426432,0.622779,0.589367,0.928384,455
1,-0.990851,-0.759214,-1.415778,1.804849,-0.262983,-0.938090,1.757228,-0.604219,-0.554667,-0.922878,-0.998922,-0.587926,-0.713687,-0.736510,0.678305,0.272933,0.146183,1355
2,1.196947,1.460959,-0.215932,-0.587848,-0.262983,-0.938090,-0.620228,1.598782,1.717040,1.216254,1.159786,-0.587926,0.142464,0.920832,-1.349029,-1.383813,-1.025934,1384
3,1.383678,1.455436,-1.084976,1.757128,-0.262983,0.568497,1.563993,-0.604219,-0.554667,1.547461,1.248794,-0.587926,-0.713687,-0.736510,0.116297,0.221120,0.392010,1404
4,-0.990851,-0.759214,0.818231,-0.587848,-0.262983,1.260252,-0.620228,-0.604219,-0.554667,-0.922878,-0.998922,-0.587926,-0.713687,-0.736510,0.996685,0.626820,0.473940,1487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,-0.990851,-0.759214,-1.415778,-0.587848,-0.262983,-0.938090,-0.620228,-0.604219,1.989558,-0.922878,-0.998922,-0.587926,-0.713687,-0.736510,-1.349029,-1.587487,-1.579448,16441
474,1.394078,-0.759214,0.818231,-0.587848,-0.262983,0.568497,1.563993,-0.604219,-0.554667,1.571138,1.240202,-0.587926,-0.713687,-0.736510,0.150160,1.519603,1.688965,16467
475,1.222938,-0.759214,0.818231,-0.587848,-0.262983,0.568497,-0.620228,-0.604219,1.886193,1.489323,1.057685,-0.587926,-0.713687,-0.736510,-1.349029,-0.301646,-0.002461,16474
476,-0.990851,-0.759214,-1.415778,1.804849,-0.262983,1.260252,-0.620228,-0.604219,-0.554667,-0.922878,-0.998922,-0.587926,-0.713687,-0.736510,2.128898,2.101688,1.838155,16476


In [45]:
# Define the desired order of feature names
feature_names_desired_order = [
    'AVG_CALL_FREQ', 'AVG_DAILY_CALLS', 'AVG_DAILY_CALLS_DUR', 'CALLTYPE_0_RATIO',
    'CALLTYPE_1_RATIO', 'CALLTYPE_6_RATIO', 'CALLTYPE_7_RATIO', 'INTERNATIONAL_RATIO',
    'MAX_CALL_FREQ', 'MAX_DAILY_CALLS', 'MAX_DAILY_CALLS_DUR', 'MIN_CALL_FREQ',
    'MIN_DAILY_CALLS', 'MIN_DAILY_CALLS_DUR', 'OFF_NET_RATIO', 'ON_NET_RATIO',
    'STD_CALL_FREQ', "A_Num"
]

# Reorder the columns of the DataFrame according to the desired feature order
transformed_df = transformed_df.reindex(columns=feature_names_desired_order)

# Make predictions using the machine learning model
predictions = model.predict(transformed_df.drop(columns=["A_Num"]))

# Create a pandas Series from the predictions with the same index as the DataFrame
predictions_series = pd.Series(predictions, index=transformed_df.index)

# Extract rows from transformed_df where the prediction is 1
A_Num_with_prediction_1 = pd.DataFrame(transformed_df.loc[predictions_series == 1, 'A_Num'])

# Create a cursor for interacting with the database
cursor = connection.cursor()

# Iterate over each A_Num with prediction 1 and insert it into the 'suspiciouses' table
for A_Num in A_Num_with_prediction_1.values:
    cursor.execute("INSERT INTO suspiciouses (subscription) VALUES (%s)", (int(A_Num[0]),))

# Commit the transaction to save the changes in the database
connection.commit()

In [46]:
# Close the cursor and connection
cursor.close()
connection.close()